Uptrail Week 1 project

In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv("C:/Users/HP/Desktop/Uptrail/Week1 Dataset/customer_signups.csv")
df.head(5)

,customer_id,name,email,signup_date,source,region,plan_selected,marketing_opt_in,age,gender
0,CUST00000,Joshua Bryant,NaN,NaN,Instagram,NaN,basic,No,34,Female
1,CUST00001,Nicole Stewart,nicole1@example.com,02-01-2024,LinkedIn,West,basic,Yes,29,Male
2,CUST00002,Rachel Allen,rachel2@example.com,03-01-2024,Google,North,PREMIUM,Yes,34,Non-Binary
3,CUST00003,Zachary Sanchez,zachary3@mailhub.org,04-01-2024,YouTube,NaN,Pro,No,40,Male
4,CUST00004,NaN,matthew4@mailhub.org,05-01-2024,LinkedIn,West,Premium,No,25,Other


In [82]:
df.count()

customer_id         298
name                291
email               266
signup_date         298
source              291
region              270
plan_selected       292
marketing_opt_in    290
age                 288
gender              292
dtype: int64

In [83]:
df.dtypes

customer_id         object
name                object
email               object
signup_date         object
source              object
region              object
plan_selected       object
marketing_opt_in    object
age                 object
gender              object
dtype: object

In [84]:
# df['signup_date'] = pd.to_datetime(df['signup_date'])
# df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
# df.dtypes


In [85]:
# bad_values = ['not a date', '', 'n/a']
# df['signup_date'] = df['signup_date'].replace(bad_values, np.nan)
# df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
# # df['signup_date'].count()
# df['signup_date'] = pd.to_datetime(df['signup_date'])
df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
# df['signup_date'] = df['signup_date'].replace('', pd.NA)
median_date = df['signup_date'].median()
df['signup_date'] = df['signup_date'].fillna(median_date)


# df['signup_date'].count()

# df['signup_date'] = pd.to_datetime(df['signup_date'], dayfirst=True, errors='coerce')
print("Valid dates:", df['signup_date'].notna().sum())
print("Invalid (NaT) dates:", df['signup_date'].isna().sum())


Valid dates: 300
Invalid (NaT) dates: 0


In [86]:
# Show rows that failed parsing
df['signup_date'].head(85)


0    2024-05-30
1    2024-01-02
2    2024-01-03
3    2024-01-04
4    2024-01-05
        ...    
80   2024-05-30
81   2024-03-22
82   2024-03-23
83   2024-03-24
84   2024-03-25
Name: signup_date, Length: 85, dtype: datetime64[ns]

In [ ]:
#converting marketing-opt_in column into boolean data type and handling missing values
df['marketing_opt_in'] = df['marketing_opt_in'].astype(str).str.lower().str.strip()
df['marketing_opt_in'] = df['marketing_opt_in'].map({
    'yes': True,
    'no' : False
})
df['marketing_opt_in'] = df['marketing_opt_in'].fillna(False)


In [ ]:
#checking for unique values in the column
df['marketing_opt_in'].unique()

array([False,  True])

In [ ]:
#checking for null values in the column 
df['marketing_opt_in'].isna().sum()

0

In [98]:
#cleaning customer_id column 
#cheking for null values
df['customer_id'].isna().sum()

2

In [99]:
#extracting numeric part and fillinh null cells as they are in sequence
df['customer_id_numeric'] = df['customer_id'].str.extract(r'(\d{5})')
df['customer_id_numeric'] = pd.to_numeric(df['customer_id_numeric'], errors='coerce')
df['customer_id_numeric'] = df['customer_id_numeric'].interpolate(method='linear')
df['customer_id_numeric'] = df['customer_id_numeric'].round().astype(int)
df['customer_id'] = df['customer_id_numeric'].apply(lambda x: f"CUST{x:05d}")

# Step 5: Drop helper column
# df.drop(columns='customer_id_numeric', inplace=True)


In [ ]:
#double chaecking for null values
df['customer_id'].isna().sum()


0

In [101]:
#Dropping helper column
df.drop(columns='customer_id_numeric', inplace=True)

In [102]:
# Handling missing name where email is available
def infer_name_from_email(email):
    if pd.isna(email):
        return np.nan
    username = email.split('@')[0]
    name_parts = ''.join([c if not c.isdigit() else ' ' + c for c in username]).split()
    return name_parts[0].capitalize() if name_parts else np.nan

df['name'] = df.apply(
    lambda row: infer_name_from_email(row['email']) if pd.isna(row['name']) else row['name'],
    axis=1
)



In [ ]:
#checking for the null values after filling names from email 
# df['name'].isna().sum()
df['name'].count()


298

In [ ]:
#cleaning source column by replacing null values by Unkown 
df['source'] = df['source'].replace(r'^\s*$|^\?\?$', np.nan, regex=True)

df['source'] = df['source'].fillna('Unknown')

df['source'] = df['source'].str.title()


In [109]:
df['source'].unique()

array(['Instagram', 'Linkedin', 'Google', 'Youtube', 'Facebook',
       'Referral', 'Unknown'], dtype=object)

In [110]:
df['source'].isna().sum()

0

In [112]:
#cleaning region column
#cheking for null and unique vlaues
df['region'].unique()

array([nan, 'West', 'North', 'South', 'Central', 'East'], dtype=object)

In [115]:
print(df['region'].count())
print(df['region'].isna().sum())

270
30


In [116]:
# Clean 'region' column
df['region'] = df['region'].replace(r'^\s*$', np.nan, regex=True)  
df['region'] = df['region'].fillna('Unknown')                      
df['region'] = df['region'].str.title()                            


In [ ]:
#rechecking the region column
print(df['region'].count())
print(df['region'].isna().sum())
print(df['region'].unique())

300
0
['Unknown' 'West' 'North' 'South' 'Central' 'East']


In [119]:
#cleaning plan_selected Column
df['plan_selected'].unique()

array(['basic', 'PREMIUM', 'Pro', 'Premium', 'UnknownPlan', 'PRO',
       'Basic', nan, 'prem'], dtype=object)

In [124]:
print(df['plan_selected'].isna().sum())
print(df['plan_selected'].value_counts(dropna=True))
print(df['plan_selected'].count())

8
plan_selected
Premium        57
Pro            53
basic          46
Basic          46
PREMIUM        42
PRO            41
UnknownPlan     6
prem            1
Name: count, dtype: int64
292


In [127]:
# dealing with null values
df['plan_selected'] = df['plan_selected'].replace(r'^\s*$', np.nan, regex=True)
df['plan_selected'] = df['plan_selected'].fillna('UnknownPlan')

# Converting all to lowercase
df['plan_selected'] = df['plan_selected'].str.lower()

# column integration/ quality
df['plan_selected'] = df['plan_selected'].replace({
    'prem': 'premium'  
})


# df['plan_selected'] = df['plan_selected'].astype('category')


In [128]:
print(df['plan_selected'].isna().sum())
print(df['plan_selected'].value_counts(dropna=True))
print(df['plan_selected'].count())

0
plan_selected
premium        100
pro             94
basic           92
unknownplan     14
Name: count, dtype: int64
300
